<a href="https://colab.research.google.com/github/PraveenKumarGarlapati/AV_LoanDelinquencyAug15th/blob/master/handling%20not%20and%20no%20words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Test

In [0]:
# useful for some cases. it takes a spacy sentence object.

def word_is_negated(word):
    """ """

    for child in word.children:
        if child.dep_ == 'neg':
            return True

    if word.pos_ in {'VERB'}:
        for ancestor in word.ancestors:
            if ancestor.pos_ in {'VERB'}:
                for child2 in ancestor.children:
                    if child2.dep_ == 'neg':
                        return True

    return False

def find_negated_wordSentIdxs_in_sent(sent, idxs_of_interest=None):
    """ """

    negated_word_idxs = set()
    for word_sent_idx, word in enumerate(sent):
        if idxs_of_interest:
            if word_sent_idx not in idxs_of_interest:
                continue
        if word_is_negated(word):
            negated_word_idxs.add(word_sent_idx)

    return negated_word_idxs

In [26]:
# !pip install spacy
# !python -m spacy download en_core_web_lg 

import spacy
nlp = spacy.load('en_core_web_sm')
find_negated_wordSentIdxs_in_sent(nlp("Hotel was not great"))

{1}

In [12]:
from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [18]:
TextBlob('This is not great').sentiment

Sentiment(polarity=-0.4, subjectivity=0.75)

In [28]:
!pip install pycorenlp
from pycorenlp import StanfordCoreNLP

  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-cp36-none-any.whl size=2143 sha256=6b1fdfc5cc3405c9d81a730a0cf800918e9abf72713ab3b5b02252ea87f64879
  Stored in directory: /root/.cache/pip/wheels/fb/e9/2f/767a7b5f2e82d587a36143c04a21839b4b14bebfb89410d2d5
Successfully built pycorenlp


In [0]:
nlp = StanfordCoreNLP('http://localhost:9000')
def get_sentiment(text):
    res = nlp.annotate(text,
                       properties={'annotators': 'sentiment',
                                   'outputFormat': 'json',
                                   'timeout': 1000,
                       })
    print(text)
    print('Sentiment:', res['sentences'][0]['sentiment'])
    print('Sentiment score:', res['sentences'][0]['sentimentValue'])
    print('Sentiment distribution (0-v. negative, 5-v. positive:', res['sentences'][0]['sentimentDistribution'])

In [31]:
get_sentiment('though i liked the place, I would never be coming back')

Exception: ignored

## Not words

In [49]:
import pandas as pd
e1 = pd.read_excel('/content/Final_0601_v1 - Copy.xlsx', sheet_name = 'AllScores', skiprows = 1)
e1.head()

,ID_SID,Sentence,entity,descriptor,All descriptors,Descriptors after extracting root words,Descriptors in 1 word neighbourhood,Descriptors in 2 words neighbourhood,Descriptors in31 words neighbourhood,Relavant Phrase,Descriptor from the phrase
0,BKG-20190696110_1,comfortable room,room,comfortable,['comfortable'],comfortable,['comfortable'],['comfortable'],['comfortable'],['comfortable room'],['comfortable']
1,BKG-20190696110_2,great location,location,great,['great'],great,['great'],['great'],['great'],['great location'],['great']
2,BKG-20190696110_3,good facilities,facilities,good,['good'],good,['good'],['good'],['good'],['good facilities'],['good']
3,BKG-201909713799_1,very good location great staff and great exper...,staff,great,"['good', 'great', 'great', 'second']","good,great,great,second",['great'],['great'],"['good', 'great']",['great staff'],['great']
4,BKG-201909713799_1,very good location great staff and great exper...,experience,great,"['good', 'great', 'great', 'second']","good,great,great,second",['great'],['great'],"['second', 'great']",['great experience'],['great']


In [44]:
#Detects Not, No etc from the sentence
e1.apply(lambda x: ",".join([i.strip() for i in x['Sentence'].split() if i in ['not','no']]), axis = 1)

0          
1          
2          
3          
4          
       ... 
1843    not
1844       
1845     no
1846       
1847       
Length: 1848, dtype: object

In [0]:
#Create a list of not words
w1 = e1['Descriptors after extracting root words'].map(str).apply(lambda x : x.split(",")).apply(pd.Series).stack().reset_index(name = 'word')

In [0]:
w1['new'] = w1.word.apply(lambda x: 'no ' + str(x))
w1['new1'] = w1.word.apply(lambda x: 'not ' + str(x))

In [0]:
desc_words = pd.concat([w1.word,w1.new,w1.new1], ignore_index = True)

In [0]:
## Pattern matching to detect bigrams

In [0]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [0]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
# the list containing the pharses to be matched
terminology_list = desc_words
# convert the phrases into document object using nlp.make_doc to #speed up.
patterns = [nlp.make_doc(text) for text in terminology_list]
# add the patterns to the matcher object without any callbacks
matcher.add("Phrase Matching", None, *patterns)
# the input text string is converted to a Document object

def phrase_matching(text):

  doc = nlp(text)
  #call the matcher object the document object and it will return #match_id, start and stop indexes of the matched words
  matches = matcher(doc)
  #print the matched results and extract out the results
  list1 = []
  for match_id, start, end in matches:
      
      # Get the string representation 
      string_id = nlp.vocab.strings[match_id]  
      span = doc[start:end]  # The matched span
      print(match_id, string_id, start, end, span.text)
      list1.append(span.text)
  return list1
      

In [0]:
# from spacy.lang.en.stop_words import STOP_WORDS

# import spacy    
# nlp = spacy.load("en")
# spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
# spacy_stopwords.remove('not')
# spacy_stopwords.remove('no')


# # e1.Sentence.apply(lambda x: [i for i in nlp(x) if not i.is_stop])

In [132]:
# e1.Sentence.head().apply(lambda x: phrase_matching(x))
phrase_matching("here iam and this is not great")

11356100181062323261 Phrase Matching 5 7 not great
11356100181062323261 Phrase Matching 6 7 great


['not great', 'great']